In [1]:
# Import all the needed libraries
import pandas as pd
import numpy as np
import re
import sys
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression


# Set options
pd.set_option('display.max_colwidth', None) 
np.set_printoptions(threshold=np.inf)
pd.options.display.max_rows = 2000
pd.options.display.max_seq_items = 2000
pd.set_option('mode.chained_assignment', None)

In [2]:
# Read the data
df = pd.read_csv("recipes.csv", encoding='utf_8')
df = df.drop_duplicates()
df.tail()

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
20047,Parmesan Puffs,3.125,28.0,2.0,2.0,64.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20048,Artichoke and Parmesan Risotto,4.375,671.0,22.0,28.0,583.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20049,Turkey Cream Puff Pie,4.375,563.0,31.0,38.0,652.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
20050,Snapper on Angel Hair with Citrus Cream,4.375,631.0,45.0,24.0,517.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20051,Baked Ham with Marmalade-Horseradish Glaze,4.375,560.0,73.0,10.0,3698.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Data Cleaning

In [3]:
# Examine the fields that are present in the data
print(df.columns)

Index(['title', 'rating', 'calories', 'protein', 'fat', 'sodium', '#cakeweek',
       '#wasteless', '22-minute meals', '3-ingredient recipes',
       '30 days of groceries', 'advance prep required', 'alabama', 'alaska',
       'alcoholic', 'almond', 'amaretto', 'anchovy', 'anise', 'anniversary',
       'anthony bourdain', 'aperitif', 'appetizer', 'apple', 'apple juice',
       'apricot', 'arizona', 'artichoke', 'arugula', 'asian pear', 'asparagus',
       'aspen', 'atlanta', 'australia', 'avocado', 'back to school',
       'backyard bbq', 'bacon', 'bake', 'banana', 'barley', 'basil', 'bass',
       'bastille day', 'bean', 'beef', 'beef rib', 'beef shank',
       'beef tenderloin', 'beer', 'beet', 'bell pepper', 'berry',
       'beverly hills', 'birthday', 'biscuit', 'bitters', 'blackberry',
       'blender', 'blue cheese', 'blueberry', 'boil', 'bok choy',
       'bon appétit', 'bon app��tit', 'boston', 'bourbon', 'braise', 'bran',
       'brandy', 'bread', 'breadcrumbs', 'breakfast', '

Observation: A lot of the columns are not going to be relevant to the analysis because they are not ingredients. From observation here are the key categories of "dirty data": <br>
1) Hashtags (starting with "#") <br>
2) Text containing numeric symbols ('22-minute meals', '3-ingredient recipes') <br>
3) Places (Country names, state names, city names) <br>
4) Events (e.g. cinco de mayo, christmas eve) <br>

One quick way to eliminate these dirty data is to use Named Entity Recognition under the spacy package, which is done in the next step:

In [4]:
# Using Named Entity Recognition to easily tag irrelevant terms 
# Subset all the relevant columns candidate for ingredient list i.e. eliminate the first few columns
pos_ingr = df.drop(columns=['title', 'rating', 'calories', 'protein', 'fat', 'sodium'], axis=1).columns

# Create a function that outputs a list of tuples containing the entity and the NER tag
def ner_tag(word):
    doc = nlp(word)
    ne = [(ent.text, ent.label_) for ent in doc.ents]
    return ne

filter = ['DATE', 'GPE', 'NORP']
filter_set = set(filter)

ner_list = [ner_tag(ner) for ner in pos_ingr if len(ner_tag(ner)) is not 0]
print(ner_list)

[[('30 days', 'DATE')], [('alabama', 'GPE')], [('alaska', 'GPE')], [('anthony bourdain', 'PERSON')], [('apple', 'ORG')], [('apple juice', 'ORG')], [('arizona', 'GPE')], [('arugula', 'PERSON')], [('asian', 'NORP')], [('atlanta', 'GPE')], [('australia', 'GPE')], [('avocado', 'ORG')], [('backyard bbq', 'PERSON')], [('bastille day', 'ORG')], [('bell', 'ORG')], [('beverly hills', 'GPE')], [('blueberry', 'PERSON')], [('bon appétit', 'PERSON')], [('bon app', 'PERSON')], [('boston', 'GPE')], [('broccoli rabe', 'ORG')], [('brooklyn', 'GPE')], [('brown rice', 'PERSON')], [('bulgaria', 'GPE')], [('burrito', 'PERSON')], [('california', 'GPE')], [('cambridge', 'GPE')], [('canada', 'GPE')], [('candy thermometer', 'PERSON')], [('chicago', 'GPE')], [('chili', 'GPE')], [('christmas', 'DATE')], [('christmas eve', 'DATE')], [('cilantro', 'GPE')], [('cinco de mayo', 'PERSON')], [('collard greens', 'PERSON')], [('colorado', 'GPE')], [('columbus', 'ORG')], [('connecticut', 'GPE')], [('costa mesa', 'ORG')], 

It is generally not a clean classification result, but the model is pretty good in identifying DATE, GPE (Geopolitical Entities), and NORP (Nationalities). The next step is to start isolating these types of entities. 

In [5]:
filter = ['DATE', 'GPE', 'NORP']
filter_set = set(filter)
ner_filtered = [tup[0][0] for tup in ner_list if tup[0][1] in filter_set]
ner_filtered

['30 days',
 'alabama',
 'alaska',
 'arizona',
 'asian',
 'atlanta',
 'australia',
 'beverly hills',
 'boston',
 'brooklyn',
 'bulgaria',
 'california',
 'cambridge',
 'canada',
 'chicago',
 'chili',
 'christmas',
 'christmas eve',
 'cilantro',
 'colorado',
 'connecticut',
 'cuba',
 'dallas',
 'denver',
 'egypt',
 'florida',
 'fourth of july',
 'france',
 'georgia',
 'germany',
 'halloween',
 'hawaii',
 'healdsburg',
 'hollywood',
 'houston',
 'idaho',
 'illinois',
 'indiana',
 'iowa',
 'ireland',
 'israel',
 'italy',
 'jamaica',
 'japan',
 'jerusalem',
 'kansas',
 'kansas',
 'kentucky',
 'kentucky',
 'las vegas',
 'london',
 'los angeles',
 'louisiana',
 'louisville',
 'maine',
 'maryland',
 'massachusetts',
 'mexico',
 'miami',
 'michigan',
 'minneapolis',
 'minnesota',
 'missouri',
 "mother's day",
 'new jersey',
 "new year's day",
 "new year's eve",
 'new york',
 'north carolina',
 'ohio',
 'oklahoma',
 'oregon',
 'paris',
 'pennsylvania',
 'peru',
 'philippines',
 'pittsburgh',
 '

In [6]:
# Clean up the list by removing obviously relevant terms which were excluded:
relevant = ['turkey','cilantro','radicchio','semolina']
ner_filtered = [e for e in ner_filtered if e not in relevant]

The next step is to isolate the other irrelevant terms using pattern matching.

In [7]:
# Isolate hashtags and words with numeric symbols
hashtag_num_filtered = list(pos_ingr[pos_ingr.str.contains('[#0-9]', regex=True)])

# Combine the lists of irrelevant keywords
irrelevant_filtered = list(set(list(ner_filtered + hashtag_num_filtered)))

# Final list of "relevant" keywords
relevant_filtered = [e for e in pos_ingr if e not in irrelevant_filtered]

We now subset the columns with the keywords that we believe are most likely "ingredients"

In [8]:
df = df[['title', 'rating', 'calories', 'protein', 'fat', 'sodium'] + relevant_filtered]

Understanding that the cleaning procedure might not have removed all the dirty keywords in the process, we do a final cleaning effort by looking at the dirty keywords with a lot of observations. <br>
In the interest of time, we'll look at the top 100 keywords and pick the obvious irrelevant ones and manually remove them from the dataset.

In [9]:
df.drop(columns=['title', 'rating', 'calories', 'protein', 'fat', 'sodium']).sum(axis=0).sort_values(ascending=False)[:101]

bon appétit          8654
peanut free          7698
soy free             7415
tree nut free        6425
vegetarian           6168
gourmet              5950
kosher               5621
pescatarian          5508
quick & easy         4853
wheat/gluten-free    4446
bake                 4109
summer               3845
dessert              3234
dairy free           2901
no sugar added       2843
side                 2776
fall                 2751
dinner               2555
sugar conscious      2218
healthy              2192
kidney friendly      2015
onion                2003
tomato               2002
vegetable            1929
sauté                1926
milk/cream           1841
fruit                1833
kid-friendly         1681
egg                  1639
vegan                1586
spring               1571
herb                 1531
garlic               1477
salad                1454
dairy                1369
appetizer            1315
lunch                1294
cheese               1292
chicken     

In [10]:
final_dirty_list = ['bon appétit', 'peanut free', 'soy free', 'tree nut free', 'vegetarian', 'gourmet', 'kosher', 
                    'pescatarian', 'quick & easy', 'wheat/gluten-free', 'bake', 'summer', 'dessert', 'dairy free', 
                    'side', 'no sugar added', 'fall', 'dinner', 'sugar conscious', 'healthy', 'kidney friendly',
                    'vegetable', 'sauté' , 'fruit', 'vegan', 'kid-friendly', 'spring', 'herb', 'salad', 'dairy', 
                    'appetizer', 'lunch', 'roast', 'no-cook', 'soup/stew', 'cocktail party', 'chill', 'grill/barbecue',
                    'drink', 'sauce', 'low cal', 'high fiber', 'food processor','backyard bbq','low fat', 'condiment/spread',     
                    'party', 'simmer', 'alcoholic','brunch', 'paleo', 'cake', 'breakfast', 'spice', 'mixer', 'poultry',
                    'citrus', 'blender', 'marinate', 'grill', 'picnic'] 

df = df.drop(columns = final_dirty_list)

## Answering the Research Questions

### 1. What ingredient has the highest probability of the recipe having a high calorie content?

To answer this, we first look at the distribution of calories. Just by looking at the descriptive statistics we can already see outliers in the data (the highest caloric content is 30M calories). 

In [11]:
df['calories'].describe()

count    1.450200e+04
mean     6.622096e+03
std      3.747819e+05
min      0.000000e+00
25%      2.050000e+02
50%      3.450000e+02
75%      5.990000e+02
max      3.011122e+07
Name: calories, dtype: float64

In [12]:
df.sort_values(by=['calories'], ascending=False)[:5]

,title,rating,calories,protein,fat,sodium,advance prep required,almond,amaretto,anchovy,...,wok,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,turkey
11391,Pear-Cranberry Mincemeat Lattice Pie,4.375,30111218.0,200968.0,1722763.0,27675110.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6368,Deep-Dish Wild Blueberry Pie,4.375,29997918.0,200210.0,1716279.0,27570999.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19672,"Apricot, Cranberry and Walnut Pie",4.375,13062948.0,87188.0,747374.0,12005810.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2976,Lamb Köfte with Tarator Sauce,5.000,4518216.0,166471.0,44198.0,7540990.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1304,"Rice Pilaf with Lamb, Carrots, and Raisins",5.000,4157357.0,236489.0,221495.0,3134853.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


I'll define a function that will remove outliers using the Tukey Rule, and then filter out the outliers from the data:

In [13]:
def tukey_rule(data_frame, column_name):
    df_column = data_frame[column_name]
    Q1 = df_column.quantile(0.25)
    Q3 = df_column.quantile(0.75)
    IQR = Q3 - Q1
    max_value = Q3 + 1.5 * IQR
    min_value = Q1 - 1.5 * IQR
    df_no_outliers = data_frame[(data_frame[column_name] < max_value) & (data_frame[column_name] > min_value)]
    return df_no_outliers

df_calorie_analysis = tukey_rule(df, 'calories')

# Check the descriptive statistics with the outliers removed:
df_calorie_analysis[df_calorie_analysis['calories'] != 0]['calories'].describe()

count    13587.000000
mean       390.475086
std        257.387981
min          1.000000
25%        197.500000
50%        322.000000
75%        538.000000
max       1188.000000
Name: calories, dtype: float64

Now we are ready to answer the research question by getting the mean calorie level per ingredient. 

In [14]:
df_calorie_vector = df_calorie_analysis['calories']
df_calorie_analysis = df_calorie_analysis.drop(columns = ['title', 'rating', 'calories', 'protein', 'fat', 'sodium'], axis=1)

# Multiply all the columns of 1 and 0 with the calorie array
df_calorie_analysis = df_calorie_analysis.mul(df_calorie_vector, axis=0)

# Replace all 0's with NAs so that they would not affect the computation of averages
cols = df_calorie_analysis.columns
df_calorie_analysis[cols] = df_calorie_analysis[cols].replace({0:np.nan})

# Get the mean across rows
df_calorie_analysis.mean(axis=0, skipna=True).sort_values(ascending=False)[:20]

crêpe                      1168.000000
mortar and pestle          1006.000000
westwood                    934.000000
new orleans                 898.000000
kitchen olympics            898.000000
epi loves the microwave     891.000000
pot pie                     843.000000
brisket                     837.105263
lamb shank                  828.250000
pork rib                    826.125000
grains                      812.600000
lamb chop                   789.478261
flat bread                  753.000000
rack of lamb                741.300000
tart                        739.500000
anthony bourdain            732.500000
guam                        732.000000
nancy silverton             718.000000
game                        717.909091
lamb                        717.829787
dtype: float64

A lot of the top keywords are irrelevant but looking at the top 20 we can see that these tend to have high caloric content:

1) crêpe <br> 
2) brisket <br>
3) lamb shank <br>
4) pork rib <br>
5) flat bread <br>

### 2. What ingredients are most important to a high rating for a recipe?

The first step is looking at the ratings distribution and dealing with the missing values. 

In [15]:
# Filter out the missing values and 0's for Rating
df_rating = df[pd.notnull(df['rating'])]
df_rating = df[df['rating'] != 0]
df_rating['rating'].describe()

count    16592.000000
mean         4.085403
std          0.663389
min          1.250000
25%          3.750000
50%          4.375000
75%          4.375000
max          5.000000
Name: rating, dtype: float64

Similar to what I did for calories, I computed for the mean rating per ingredient and pulled up the top ingredients with the highest average rating.

In [16]:
df_rating_vector = df_rating['rating']
df_rating = df_rating.drop(columns = ['title', 'rating', 'calories', 'protein', 'fat', 'sodium'], axis=1)

# Multiply all the columns of 1 and 0 with the rating array
df_rating = df_rating.mul(df_rating_vector, axis=0)

# Replace all 0's with NAs so that they would not affect the computation of averages
cols = df_rating.columns
df_rating[cols] = df_rating[cols].replace({0:np.nan})

# Get the mean across rows
df_rating.mean(axis=0, skipna=True).sort_values(ascending=False)[:20]

mortar and pestle    5.000000
flat bread           5.000000
suzanne goin         5.000000
sourdough            5.000000
cookbook critic      5.000000
cupcake              5.000000
custard              5.000000
emeril lagasse       5.000000
sardine              5.000000
brownie              5.000000
rub                  5.000000
burrito              5.000000
pickles              5.000000
yonkers              5.000000
juicer               4.791667
granola              4.687500
grappa               4.687500
omelet               4.687500
soufflé/meringue     4.687500
lychee               4.687500
dtype: float64

Here are the top ingredients from dishes with the highest average rating:

1) flat bread 
2) sourdough
3) cupcake
4) custard
5) sardine

### 3)  If I typically enjoy high protein to fat ratio, which recipes should I try?

The first step is to remove the outliers for both protein and fat metrics using the function that we created before.

In [17]:
df_protein_fat_analysis = df[(df['protein'] != 0) & (df['fat'] != 0) & (pd.notnull(df['protein'])) & (pd.notnull(df['fat']))]

df_protein_fat_analysis = tukey_rule(df_protein_fat_analysis, 'protein')
df_protein_fat_analysis = tukey_rule(df_protein_fat_analysis, 'fat')

# Check the descriptive statistics with the outliers removed:
df_protein_fat_analysis['protein'].describe()

count    11627.000000
mean        15.406296
std         15.605555
min          1.000000
25%          4.000000
50%          9.000000
75%         23.000000
max         71.000000
Name: protein, dtype: float64

In [18]:
df_protein_fat_analysis['fat'].describe()

count    11627.000000
mean        20.673347
std         14.726497
min          1.000000
25%          9.000000
50%         17.000000
75%         29.000000
max         64.000000
Name: fat, dtype: float64

Once the outliers are removed, I can now compute for the protein to fat ratio 

In [19]:
df_protein_fat_analysis['protein_fat_ratio'] = df_protein_fat_analysis['protein'] / df_protein_fat_analysis['fat']
df_protein_fat_analysis['protein_fat_ratio'].describe()

count    11627.000000
mean         0.951939
std          1.210858
min          0.016393
25%          0.306624
50%          0.631579
75%          1.125000
max         30.000000
Name: protein_fat_ratio, dtype: float64

The following recipes have the highest protein to fat ratio:

In [20]:
df_protein_fat_analysis.sort_values(by =['protein_fat_ratio'], ascending=False)[['title','protein_fat_ratio']][:5]

,title,protein_fat_ratio
13867,Fish and Yuca Stew with Pickled Onions,30.0
10855,Lobster Gelees with Fresh Tarragon Oil,27.5
281,Fish Mixed Grill,22.5
18646,Sweet and Sour Crab Salad,22.0
9266,"Scallop, Shrimp, and Squid ""Ceviche""",16.0


### 4) If you had to pick 10 recipes to recommend based on this data, what would you recommend?

I think that there are two important metrics to look at to answer this question: ratings and calorie count. It's great to have a low-calorie meal that's well-reviewed (meaning more likely to be satisfying). 

My approach is to form a ratio of ratings to calorie count - the higher the score in this metric, the better it is. 

One thing to watch out for is that ratings and calorie count are on a very different scale. My approach for this is to get the percentile rank for each metric first, and then compute for the ratio. 

In [21]:
df_rating_calorie = df[(df['rating'] != 0) & (df['calories'] != 0) & (pd.notnull(df['rating'])) & (pd.notnull(df['calories']))]
df_rating_calorie = tukey_rule(df, 'calories')

# Compute for percentile ranks
df_rating_calorie['pct_rating'] = df_rating_calorie['rating'].rank(pct=True)
df_rating_calorie['pct_calories'] = df_rating_calorie['calories'].rank(pct=True)

# Compute for rating-calorie ratio
df_rating_calorie['rating_calorie_ratio'] = df_rating_calorie['pct_rating'] / df_rating_calorie['pct_calories']
df_rating_calorie['rating_calorie_ratio'].describe()

count    13594.000000
mean         3.907401
std         43.968667
min          0.042046
25%          0.514264
50%          0.966821
75%          1.852456
max       3186.125000
Name: rating_calorie_ratio, dtype: float64

Here are the top 10 recommended recipes based on rating-to-calorie ratio:

In [22]:
df_rating_calorie.sort_values(by =['rating_calorie_ratio'], ascending=False)[['title','rating_calorie_ratio']][:10]

,title,rating_calorie_ratio
4848,To Clarify Butter,3186.125000
6089,Dashi (Japanese Sea Stock),3186.125000
285,Salted Water for Boiling,1122.125000
5324,Wakame,849.633333
17662,Dashi,606.166667
615,Za'atar,480.924528
14443,Brain-Boosting Broth,480.924528
13967,The Manhattan,472.473684
7321,Garlic Broth,472.473684
15115,Fresh Vegetable Platter with Olive Oil Dip,472.473684


### 5) What other questions can we seek to answer given this data?

The following can be done for further analysis:
    
1) What is the most optimal combination of ingredient that would lead to highest rating? (Can be answered through market basket analysis) <br>
2) What are the key recipe categories / clusters? (Can be answered through cluster analysis) <br>
3) Which ingredients influence the ratings / caloric content the most? (Can be answered through modeling techniques such as regression / classification / machine learning)

Furthermore, we can do a more effective cleaning of the data through modeling, or researching for better ingredient libraries.